# Capstone Project - The Battle of the Neighborhoods (Week 2)
### COURSERA IBM Applied Data Science Capstone Project


## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for an Azerbaijani restaurant in Amsterdam. Specifically, this report will be targeted to stakeholders interested in opening an **Restaurant specialized in Azerbaijani cuisine** in **Amsterdam**, Netherlands.

Since there are lots of international restaurant in Amsterdam we will try to detect **locations that are not already crowded with restaurant from Azerbaijan region**. Since Azerbaijani culture consist of Turkish and Caucasian aspects as well, we will consider both 3 category restaurant in investigation area. We are also particularly interested in **areas with no restaurants in vicinity**. We would also prefer locations **as close to city center as possible**, assuming that first two conditions are met.

We will use our data science powers to generate a few most promising neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

In [38]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files


!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [39]:
ams_df = pd.read_csv('Amsterdam_Neighbourhoods.csv')

In [40]:
print(ams_df.shape)
ams_df.head()

(105, 3)


,Neighbourhoods,Latitude,Longitude
0,Admiralenbuurt,52.372734,4.856363
1,Amsteldorp,52.360540,4.905160
2,Amsterdam Oud-West,52.365390,4.870220
3,Amsterdam Oud-Zuid,52.352350,4.877880
4,Amsterdam Science Park,52.354320,4.958030


In [41]:
print('The dataframe has {} neighborhoods and {} values.'.format(
        len(ams_df['Neighbourhoods'].unique()),
        ams_df.shape[0]
    )
)

The dataframe has 105 neighborhoods and 105 values.


In [42]:
address = 'Amsterdam'

geolocator = Nominatim(user_agent="am_explorer")
location = geolocator.geocode(address)
lat_ams = location.latitude
long_ams = location.longitude
print('The geograpical coordinate of Amsterdam are {}, {}.'.format(lat_ams, long_ams))

The geograpical coordinate of Amsterdam are 52.3727598, 4.8936041.


## Getting Venues Data using Foursquare

In [13]:
CLIENT_ID = 'T5YEPDCDF5UOJZK5LA0BYIJM25DMERSNRWJI3GT0VUMUQYIY' # your Foursquare ID
CLIENT_SECRET = 'ICKVICYAY1R5BQXKOGWFG0TEI5PMHPN23J2T0J3GEBDXLD4S' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: T5YEPDCDF5UOJZK5LA0BYIJM25DMERSNRWJI3GT0VUMUQYIY
CLIENT_SECRET:ICKVICYAY1R5BQXKOGWFG0TEI5PMHPN23J2T0J3GEBDXLD4S


In [14]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [43]:
#Get venues for all neighborhoods in our dataset
ams_venues = pd.read_csv('ams_venues.csv')
#getNearbyVenues(names = ams_df['Neighbourhoods'],
                                #latitudes = ams_df['Latitude'],
                                #longitudes = ams_df['Longitude']
                            #)

In [ ]:
# save the extracted data as csv file

#ams_venues.to_csv('ams_venues.csv')
#ams_venues.save('ams_venues.csv')

In [44]:
#Check size of resulting dataframe
print(ams_venues.shape)
ams_venues.head()

(4150, 8)


,Unnamed: 0,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,0,Admiralenbuurt,52.372734,4.856363,Radijs,52.371049,4.856756,Bistro
1,1,Admiralenbuurt,52.372734,4.856363,Rein Cityspa,52.371217,4.855969,Spa
2,2,Admiralenbuurt,52.372734,4.856363,Sapporo Ramen Sora,52.371294,4.855144,Ramen Restaurant
3,3,Admiralenbuurt,52.372734,4.856363,Maz Mez,52.371231,4.857968,Lebanese Restaurant
4,4,Admiralenbuurt,52.372734,4.856363,Café Cook,52.371208,4.852792,Pub


In [45]:
#Number of venues per neighborhood
ams_venues.groupby('Neighborhood').count()

,Unnamed: 0,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,
Admiralenbuurt,44,44,44,44,44,44,44
Amsteldorp,58,58,58,58,58,58,58
Amsterdam Oud-West,92,92,92,92,92,92,92
Amsterdam Oud-Zuid,57,57,57,57,57,57,57
Amsterdam Science Park,6,6,6,6,6,6,6
Apollobuurt,29,29,29,29,29,29,29
Betondorp,9,9,9,9,9,9,9
Bijlmermeer,4,4,4,4,4,4,4
Binnenstad,62,62,62,62,62,62,62


In [46]:
#Number of unique venue categories
print('There are {} uniques categories.'.format(len(ams_venues['Venue Category'].unique())))

There are 286 uniques categories.


In [47]:
# check if the results contain "Caucasian Restaurant"
"Caucasian Restaurant" in ams_venues['Venue Category'].unique()

True

In [48]:
# check if the results contain "Azerbaijani Restaurant"
"Azerbaijani Restaurant" in ams_venues['Venue Category'].unique()

False

In [49]:
# check if the results contain "Turkish Restaurant"
"Turkish Restaurant" in ams_venues['Venue Category'].unique()

True

__Analyze Each Neighborhood__

In [50]:
# one hot encoding
to_onehot = pd.get_dummies(ams_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
to_onehot['Neighborhood'] = ams_venues['Neighborhood'] 

# move neighbourhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

print(to_onehot.shape)
to_onehot.head()

(4150, 286)


,Zoo Exhibit,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Baseball Field,Bath House,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Canal,Canal Lock,Candy Store,Caribbean Restaurant,Casino,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cruise,Cupcake Shop,Dairy Store,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore,Dutch Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Fraternity House,French Restaurant,Fried Chicken Joint,Friterie,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Historic Site,History Museum,Hobby Shop,Hockey Field,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Latin American Restaurant,Laundry Service,Lawyer,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Museum,Music School,Music Venue,Nature Preserve,Neighborhood,Nightclub,Noodle House,North Indian Restaurant,Notary,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Photography Studio,Pier,Pilates Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Pop-Up Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Science Museum,Seafood Restaurant,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Spiritual Center,Sporting Goods Shop,Sports Club,Stables,Stadium,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Windmill,Wine Bar,Wine Shop,Women's St

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [51]:
to_grouped = to_onehot.groupby(["Neighborhood"]).mean().reset_index()

print(to_grouped.shape)
to_grouped

(105, 286)


,Neighborhood,Zoo Exhibit,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Baseball Field,Bath House,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Canal,Canal Lock,Candy Store,Caribbean Restaurant,Casino,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cruise,Cupcake Shop,Dairy Store,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore,Dutch Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Fraternity House,French Restaurant,Fried Chicken Joint,Friterie,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Historic Site,History Museum,Hobby Shop,Hockey Field,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Latin American Restaurant,Laundry Service,Lawyer,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Museum,Music School,Music Venue,Nature Preserve,Nightclub,Noodle House,North Indian Restaurant,Notary,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Outdoors & Recreation,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Photography Studio,Pier,Pilates Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Pop-Up Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Restaurant,Road,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Science Museum,Seafood Restaurant,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Spiritual Center,Sporting Goods Shop,Sports Club,Stables,Stadium,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Windmill,Wine Bar,Wine Shop,Women's St

In [52]:
len(to_grouped[to_grouped["Caucasian Restaurant"] > 0])

1

In [53]:
len(to_grouped[to_grouped["Turkish Restaurant"] > 0])

22

Create a new dataframe to find "Turkish Restaurant" only, since "Caucasian Restaurant" category consist of  just 1 venue

In [54]:
am_turk = to_grouped[["Neighborhood","Turkish Restaurant"]]

In [55]:
am_turk.head()

,Neighborhood,Turkish Restaurant
0,Admiralenbuurt,0.0
1,Amsteldorp,0.0
2,Amsterdam Oud-West,0.0
3,Amsterdam Oud-Zuid,0.0
4,Amsterdam Science Park,0.0


## Cluster Neighborhood

Run k-means to cluster the neighborhoods in Amsterdam into 3 clusters.

In [56]:
# set number of clusters
toclusters = 3

to_clustering = am_turk.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=0).fit(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0], dtype=int32)

In [57]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
to_merged = am_turk.copy()

# add clustering labels
to_merged["Cluster Labels"] = kmeans.labels_

In [58]:
to_merged.head()

,Neighborhood,Turkish Restaurant,Cluster Labels
0,Admiralenbuurt,0.0,1
1,Amsteldorp,0.0,1
2,Amsterdam Oud-West,0.0,1
3,Amsterdam Oud-Zuid,0.0,1
4,Amsterdam Science Park,0.0,1


In [59]:

# merge grouped with data to add latitude/longitude for each neighborhood
to_merged = to_merged.join(ams_df.set_index("Neighbourhoods"), on="Neighborhood")

print(to_merged.shape)
to_merged.head()

(105, 5)


,Neighborhood,Turkish Restaurant,Cluster Labels,Latitude,Longitude
0,Admiralenbuurt,0.0,1,52.372734,4.856363
1,Amsteldorp,0.0,1,52.360540,4.905160
2,Amsterdam Oud-West,0.0,1,52.365390,4.870220
3,Amsterdam Oud-Zuid,0.0,1,52.352350,4.877880
4,Amsterdam Science Park,0.0,1,52.354320,4.958030


In [60]:
# sort the results by Cluster Labels
print(to_merged.shape)
to_merged.sort_values(["Cluster Labels"], inplace=True)
to_merged

(105, 5)


,Neighborhood,Turkish Restaurant,Cluster Labels,Latitude,Longitude
51,Museumkwartier,0.043478,0,52.362290,4.923910
83,Spaarndammerbuurt,0.022727,0,52.388686,4.879933
84,Staatsliedenbuurt,0.031250,0,52.380050,4.871490
19,Dapperbuurt,0.044444,0,52.362774,4.927368
64,Oostpoort,0.025000,0,52.356740,4.927790
88,Transvaalbuurt,0.043478,0,52.393080,4.645550
94,Watergraafsmeer,0.047619,0,52.354280,4.929670
9,Bos en Lommer,0.037736,0,52.379190,4.851740
62,Oosterparkbuurt,0.057692,0,52.359470,4.922980
55,Nieuwe Pijp,0.045455,0,52.361996,4.799175


In [61]:
# create map
map_clusters = folium.Map(location=[lat_ams, long_ams], zoom_start=12)

# set color scheme for the clusters
x = np.arange(toclusters)
ys = [i+x+(i*x)**2 for i in range(toclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_merged['Latitude'], to_merged['Longitude'], to_merged['Neighborhood'], to_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster))
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [47]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

## Examine Clusters

In [36]:
#Cluster 0
to_merged.loc[to_merged['Cluster Labels'] == 0]

,Neighborhood,Turkish Restaurant,Cluster Labels,Latitude,Longitude
51,Museumkwartier,0.043478,0,52.362290,4.923910
83,Spaarndammerbuurt,0.022727,0,52.388686,4.879933
84,Staatsliedenbuurt,0.031250,0,52.380050,4.871490
19,Dapperbuurt,0.044444,0,52.362774,4.927368
64,Oostpoort,0.025000,0,52.356740,4.927790
88,Transvaalbuurt,0.043478,0,52.393080,4.645550
94,Watergraafsmeer,0.047619,0,52.354280,4.929670
9,Bos en Lommer,0.037736,0,52.379190,4.851740
62,Oosterparkbuurt,0.057692,0,52.359470,4.922980
55,Nieuwe Pijp,0.045455,0,52.361996,4.799175


In [37]:
#Cluster 1
to_merged.loc[to_merged['Cluster Labels'] == 1]

,Neighborhood,Turkish Restaurant,Cluster Labels,Latitude,Longitude
58,Olympisch Kwartier,0.000000,1,52.369930,4.907880
73,Plantage,0.000000,1,52.366620,4.911890
72,Overtoomse Veld,0.000000,1,52.363540,4.836800
71,Overtoombuurt,0.000000,1,52.359582,4.862307
70,Overhoeks,0.000000,1,52.391680,4.905780
53,Negen Straatjes,0.000000,1,52.369930,4.907880
54,Nieuw Sloten,0.000000,1,52.345610,4.811920
69,Oude Pijp,0.000000,1,52.356250,4.890570
65,Oostzanerwerf,0.000000,1,52.425365,4.885640
63,Oostoever,0.000000,1,52.370000,4.830460


In [50]:
#Cluster 2
to_merged.loc[to_merged['Cluster Labels'] == 2]

,Neighborhood,Turkish Restaurant,Cluster Labels,Latitude,Longitude
66,Osdorp,0.111111,2,52.356360,4.790600
67,Oud Osdorp,0.111111,2,52.356360,4.790600
68,Oud-Oost,0.081081,2,52.360130,4.925320
10,Buiksloot,0.100000,2,52.408070,4.918960
45,Kolenkit District,0.068966,2,52.377660,4.842810
81,Slotermeer,0.093750,2,52.381041,4.821146


## Observations

Most of the Turkish restaurants are in Cluster 0 and Cluster 1 neighbourhoods which are Amsterdam-West and Amsterdam-Oost areas. Should highlight, there are good opportunities to open in Cluster 1 neighbourhood - Overtoombuurt. The area if good since the competition seems to be low and attractivenes of the area is high. Thus, this project recommends stakeholders to open an authentic Turkish restaurant in these locations with little to no competition and introduce Azerbaijani cuisine as uniqueness of the venue. Nonetheless, if the food is authentic, affordable and good taste, I am confident that it will have great following everywhere